In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
!pip install supabase
from supabase import create_client
from kaggle_secrets import UserSecretsClient
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
df = pd.read_csv("/kaggle/input/train-csv-hackkbyte3-0/train-2.csv")

In [4]:
df.head()

,Issue_id,Component,Title,Description,Status,Resolution,Priority
0,1700,Layout,font size 9px rounded down to 8px {font} {ll},There is a piece of code in the Windows GFX th...,VERIFIED,FIXED,0
1,7991,Plug-ins,Land OOP plugins in mozilla-central (preffed off),We need to land OOP plugins in mozilla-central...,RESOLVED,FIXED,0
2,1150,XPCOM,XP-COM Proxy code crashes when trying to AutoP...,(This bug imported from BugSplat; Netscapes in...,VERIFIED,FIXED,0
3,6143,XUL,crash setting display:none on grid rows [@ nsG...,User-Agent: Mozilla/5.0 (Windows; U; Win...,VERIFIED,FIXED,0
4,5952,Networking: Cache,limit the memory cache to a reasonable value,I have 1GB of RAM; so Im using 31MB of memory ...,VERIFIED,FIXED,0


In [5]:
print("id:" + str(df['Issue_id'][100]))
print("component:" + str(df['Component'][100]))
print("description:" + str(df['Description'][100]))

id:943
component:DOM: Core & HTML
description:Window.open() method Crashes Viewer as well as apprunner.; ; There are already somebugs about it. Bug 1098 is similar to this but for Windows; NT platform and already marked as duplicate of bug 1252. Actually bug 1252 is; little different than this bug; and bug 1252 is already marked as resolved and; fixed. I guess it would be better to report seperate bug this problem since; platform is also different and bug is also little different.; ; Product: Seamonkey [Apprunner as well as viewer]; Build: 04-22-17.; OS: Win-95 as well as MacOS.; ; Steps to Reproduce:; 1] Please copy the code Im providing.; 2] Save it as HTML file on your local machine.; 3] Open this HTML file in viewer as well as apprunner.; 4] There you will find on button called Open New Window. click this button.; ; Expected Results : Application should open new window.; ; Actual Results:; A] Viewer: Application crashes immediately.; B] Apprunner: Application opens new window; but 

In [8]:
print("id:" + str(df['Issue_id'][201]))
print("component:" + str(df['Component'][201]))
print("description:" + str(df['Description'][201]))

id:5687
component:DOM: Core & HTML
description:User-Agent:       Mozilla/5.0 (Windows; U; Windows NT 5.0; en-US; rv:1.7.5) Gecko/20041107 Firefox/1.0; Build Identifier: Mozilla/5.0 (Windows; U; Windows NT 5.0; en-US; rv:1.7.5) Gecko/20041107 Firefox/1.0; ; If the window.open() function is used to create a popup from within a frameset;; and a link back to a target frame in the parent frameset is dynamically written; using the document.writeln() function then the URL opens in a new window and; not in the targeted frame.  If the popup is actually a pre-written page on the; server then it all works as expected.  This behaviour does not happen in Mozilla; (Mozilla/5.0 (Windows; U; Windows NT 5.0; en-US; rv:1.7) Gecko/20040616) or IE6.; See Also http://forums.mozillazine.org/viewtopic.php?p=974823#974823; ; Reproducible: Always; Steps to Reproduce:; 1.Create a frameset; 2.in one of the frame windows use some JS with window.open() and; document.writeln() to create a popup with a link which is

In [12]:
train_df, val_df = train_test_split(df, test_size=0.25, random_state=7,stratify=df['Priority'])

In [13]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
def filter_data(df,components, titles, descriptions,max_length=512):
    df["combined_texts"] = [f"{component} [SEP] {title} [SEP] {description}" 
                      for component, title, description in zip(components, titles, descriptions)]
    df['token_count'] = df['combined_texts'].apply(lambda x: len(tokenizer.encode(str(x), add_special_tokens=True)))
    
    # Keep only rows with <= 512 tokens
    df_filtered = df[df['token_count'] <= 512].copy()
    df_filtered = df_filtered.drop('combined_texts', axis=1)
    return df_filtered
    
    # return inputs
train_inputs=filter_data(train_df,train_df["Component"],train_df["Title"],train_df["Description"])

Token indices sequence length is longer than the specified maximum sequence length for this model (1589 > 512). Running this sequence through the model will result in indexing errors


In [15]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("NEXT_PUBLIC_SUPABASE_ANON_KEY")
secret_value_1 = user_secrets.get_secret("NEXT_PUBLIC_SUPABASE_URL")
# Setup
url = secret_value_1
key = secret_value_0
supabase = create_client(url, key)


In [27]:
pids=[]
projects = supabase.table("projects").select("id").execute()
for i in projects.data:
    # print(type(i[1]))
    # print(i[1])
    pids.append(i["id"])
pids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [29]:
uids = []
users = supabase.table("users").select("id").eq("role", "Tester").execute()
for i in users.data:
    uids.append(i["id"])
uids


['003e71bf-b332-4c44-aec0-b692138deddf',
 '5b45e8ca-02fd-4bcc-a178-f79058b6c742']

In [30]:
embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
# Helper functions
def combine_document_features(document):
    return f"Bug ID #{document['Issue_id']} in component {document['Component']} with Description {(document['Description'])}"

# this is for new entry
def generate_embedding(row):
    doc_text = combine_document_features(row)
    embeddings = embedding_model.encode([doc_text])[0].tolist()
    embedding_str = str(embeddings)
    return embedding_str

In [32]:
# Assuming train_df is already defined and sample_data contains the "created_by" field value
import random
for i in range(1000):
    row = {
        "id": int(train_df["Issue_id"].iloc[i]),
        "title": str(train_df["Title"].iloc[i]),
        "description": str(train_df["Description"].iloc[i]),
        "status": str(train_df["Status"].iloc[i]),
        "priority_level": int(train_df["Priority"].iloc[i]),
        "created_by": (uids[random.randint(0,len(uids)-1)]),  # Assuming sample_data is a single string/user ID
        "component": str(train_df["Component"].iloc[i]),
        "tag": str(train_df["Resolution"].iloc[i]),
        "project_id": pids[random.randint(0,len(pids)-1)],
        "embeddings":generate_embedding(train_df.iloc[i])
    }
    response = supabase.table("bugs").insert(row).execute()



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# response = supabase.table("bugs").delete().neq("id", -1).execute()